In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [13]:
#Reading dataset
df_WT_Phoenix_2013=pd.read_excel(open('Phoenix climate date_2013_ 2015.xlsx', 'rb'),sheet_name='Phoenix 2013') 
df_WT_Phoenix_2013=df_WT_Phoenix_2013.rename(columns={"Unnamed: 0": "Hour"})
df_WT_Phoenix_2013=df_WT_Phoenix_2013.iloc[:,:16]

#Cleaning dataset
df_WT_Phoenix_2013=df_WT_Phoenix_2013.dropna(axis=1, thresh=len(df_WT_Phoenix_2013['Hour'].values)*0.80);
df_WT_Phoenix_2013=df_WT_Phoenix_2013.drop(columns=['Hour','Temp (C )','sind_spd (m/s)','date'])

#3 days dataset ready
X=df_WT_Phoenix_2013.iloc[:,:-1]
Y=df_WT_Phoenix_2013.iloc[2:,-1]
le = preprocessing.LabelEncoder()
D2=X.iloc[:-2,:];
D1=X.iloc[1:-1,:];
D0=X.iloc[2:,:];

#Columns create
Column_DBY=[];
Column_Y=[];
Column_T=[];
Col=D2.columns;
for col in Col:
    Column_DBY.append('DBY_'+str(col))
    Column_Y.append('Y_'+str(col))
    Column_T.append('T_'+str(col))
dictionary_DBY = dict(zip(D2.columns, Column_DBY))   
dictionary_Y = dict(zip(D1.columns, Column_Y)) 
dictionary_T = dict(zip(D0.columns, Column_T)) 

#New DataFrame Ready
DF=pd.concat([D2.rename(columns=dictionary_DBY).reset_index(),D1.rename(columns=dictionary_Y).reset_index(),D0.rename(columns=dictionary_T).reset_index()],axis=1)
DF=DF.drop(columns=['index']);

In [ ]:
D2=X.iloc[:-2,:];
D1=X.iloc[1:-1,:];
D0=X.iloc[2:,:];

#Columns create
Column_DBY=[];
Column_Y=[];
Column_T=[];
Col=D2.columns;
for col in Col:
    Column_DBY.append('DBY_'+str(col))
    Column_Y.append('Y_'+str(col))
    Column_T.append('T_'+str(col))
dictionary_DBY = dict(zip(D2.columns, Column_DBY))   
dictionary_Y = dict(zip(D1.columns, Column_Y)) 
dictionary_T = dict(zip(D0.columns, Column_T)) 

#New DataFrame Ready
DF=pd.concat([D2.rename(columns=dictionary_DBY).reset_index(),D1.rename(columns=dictionary_Y).reset_index(),D0.rename(columns=dictionary_T).reset_index()],axis=1)
DF=DF.drop(columns=['index']);

In [14]:
R1=['Partly Cloudy', 'Scattered Clouds', 'Mostly Cloudy', 'Overcast'] #Cloudy
R2=['Haze', 'Widespread Dust', 'Unknown'] #Other
R3=['Rain', 'Thunderstorm', 'Thunderstorms and Rain', 'Thunderstorms with Hail','Heavy Rain','Heavy Thunderstorms and Rain','Light Rain','Light Thunderstorms and Rain'];
R=[R1,R2,R3]
Out=['Cloudy','Other','Rainy']
for i in range(len(R)):
    for j in range(len(R[i])):
        Y=Y.replace(R[i][j],Out[i]);    

In [23]:
y=le.fit_transform(Y)

In [24]:
#One Hot encoding
DF=DF.replace(0,np.nan);
DF_cat=pd.get_dummies(DF,columns=['DBY_dir','Y_dir','T_dir']);
#Imputing
fill_NaN = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_DF = pd.DataFrame(fill_NaN.fit_transform(DF_cat))
imputed_DF.columns = DF_cat.columns
imputed_DF.index = DF_cat.index

In [26]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
imputed_std=std.fit_transform(imputed_DF)

In [28]:
#Data Splitting and model fitting
X_train, X_test, y_train, y_test = train_test_split(imputed_std, y, test_size=0.21, random_state=1)

gnb=GaussianNB();

gnb=gnb.fit(X_train,y_train)

y_pred = gnb.predict(X_train)

#Accuracy
accuracy_score(y_pred,y_train)

0.2290423415743109

In [29]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [30]:
lr=lr.fit(X_train,y_train)

y_pred = lr.predict(X_train)

#Accuracy
accuracy_score(y_pred,y_train)

C:\Users\rezwa\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\rezwa\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7158283603296391